In [ ]:
# Install the packages
%pip install openai azure-ai-evaluation azure-identity promptflow-azure

In [ ]:
import os

os.environ["AZURE_DEPLOYMENT_NAME"] = "Ravi-OpenAI-Inst-032523"
os.environ["AZURE_ENDPOINT"] = "https://ravi-openai-inst-032523.openai.azure.com/"
os.environ["AZURE_API_VERSION"] = "gpt-35-turbo"
os.environ["AZURE_SUBSCRIPTION_ID"] = "a234e53c-b063-4702-bcf4-9457e53c7af2"
os.environ["AZURE_RESOURCE_GROUP"] = "Ravi_OpenAI032523"
os.environ["AZURE_PROJECT_NAME"] = "safety_eval"

In [ ]:
from pprint import pprint
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import ProtectedMaterialEvaluator, IndirectAttackEvaluator
from azure.ai.evaluation.simulator import AdversarialSimulator, AdversarialScenario, IndirectAttackSimulator
from openai import AzureOpenAI


azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_RESOURCE_GROUP"),
    "project_name": os.environ.get("AZURE_PROJECT_NAME"),
}

credential = DefaultAzureCredential()

In [ ]:
from typing import List, Dict, Optional


async def protected_material_callback(
    messages: List[Dict], stream: bool = False, session_state: Optional[str] = None, context: Optional[Dict] = None
) -> dict:
    deployment = os.environ.get("AZURE_DEPLOYMENT_NAME")
    endpoint = os.environ.get("AZURE_ENDPOINT")
    token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
    # Get a client handle for the model
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_version=os.environ.get("AZURE_API_VERSION"),
        azure_ad_token_provider=token_provider,
    )
    # Call the model
    completion = client.chat.completions.create(
        model=deployment,
        messages=[
            {
                "role": "user",
                "content": messages["messages"][0]["content"],  # injection of prompt happens here.
            }
        ],
        max_tokens=800,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
    )

    formatted_response = completion.to_dict()["choices"][0]["message"]
    messages["messages"].append(formatted_response)
    return {
        "messages": messages["messages"],
        "stream": stream,
        "session_state": session_state,
        "context": context,
    }

In [ ]:
# initialize the adversarial simulator
protected_material_simulator = AdversarialSimulator(azure_ai_project=azure_ai_project, credential=credential)


protected_material_scenario = AdversarialScenario.ADVERSARIAL_CONTENT_PROTECTED_MATERIAL

In [ ]:
unfiltered_protected_material_outputs = await protected_material_simulator(
    scenario=protected_material_scenario,
    max_conversation_turns=3,  # define the number of conversation turns
    max_simulation_results=10,  # define the number of simulation results
    target=protected_material_callback,  # define the target model callback
)

In [ ]:
# Results are truncated for brevity.
truncation_limit = 50
for output in unfiltered_protected_material_outputs:
    for turn in output["messages"]:
        print(f"{turn['role']} : {turn['content'][0:truncation_limit]}")

In [ ]:
from pathlib import Path

print(unfiltered_protected_material_outputs.to_eval_qr_json_lines())
output = unfiltered_protected_material_outputs.to_eval_qr_json_lines()
file_path = "unfiltered_protected_material_output.jsonl"

# Write the output to the file
with Path.open(Path(file_path), "w") as file:
    file.write(output)

In [ ]:
protected_material_eval = ProtectedMaterialEvaluator(azure_ai_project=azure_ai_project, credential=credential)

result = evaluate(
    data=file_path,
    evaluators={"protected_material": protected_material_eval},

    azure_ai_project=azure_ai_project,

    output_path="./mynewfilteredIPevalresults.json",
)

In [ ]:
filtered_protected_material_outputs = await protected_material_simulator(
    scenario=protected_material_scenario,
    max_conversation_turns=3,  # define the number of conversation turns
    max_simulation_results=10,  # define the number of simulation results
    target=protected_material_callback,  
)

In [ ]:
print(filtered_protected_material_outputs.to_eval_qr_json_lines())
output = filtered_protected_material_outputs.to_eval_qr_json_lines()
filtered_protected_material_file_path = "filtered_protected_material_output.jsonl"

# Write the output to the file
with Path.open(Path(filtered_protected_material_file_path), "w") as file:
    file.write(output)

In [ ]:
filtered_result = evaluate(
    data=filtered_protected_material_file_path,
    evaluators={"protected_material": protected_material_eval},

    azure_ai_project=azure_ai_project,

    output_path="./myfilteredevalresults.json",
)

In [ ]:
from typing import List, Dict, Optional


async def xpia_callback(
    messages: List[Dict], stream: bool = False, session_state: Optional[str] = None, context: Optional[Dict] = None
) -> dict:
    messages_list = messages["messages"]
    # get last message
    latest_message = messages_list[-1]
    query = latest_message["content"]
    context = None
    if "file_content" in messages["template_parameters"]:
        query += messages["template_parameters"]["file_content"]


    # Get a client handle for the model
    deployment = os.environ.get("AZURE_DEPLOYMENT_NAME")

    token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

    oai_client = AzureOpenAI(
        azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
        api_version=os.environ.get("AZURE_API_VERSION"),
        azure_ad_token_provider=token_provider,
    )
    try:
        response_from_oai_chat_completions = oai_client.chat.completions.create(
            messages=[{"content": query, "role": "user"}], model=deployment, max_tokens=300
        )
        print(response_from_oai_chat_completions)
    except Exception as e:
        print(f"Error: {e} with content length {len(query)}")
        # to continue the conversation, return the messages, else you can fail the adversarial with an exception
        message = {
            "content": "Something went wrong. Check the exception e for more details.",
            "role": "assistant",
            "context": None,
        }
        messages["messages"].append(message)
        return {"messages": messages["messages"], "stream": stream, "session_state": session_state}
    response_result = response_from_oai_chat_completions.choices[0].message.content
    formatted_response = {
        "content": response_result,
        "role": "assistant",
        "context": {},
    }
    messages["messages"].append(formatted_response)
    return {"messages": messages["messages"], "stream": stream, "session_state": session_state, "context": context}

In [ ]:
indirect_attack_simulator = IndirectAttackSimulator(
    azure_ai_project=azure_ai_project, credential=DefaultAzureCredential()
)

unfiltered_indirect_attack_outputs = await indirect_attack_simulator(
    target=xpia_callback,
    scenario=AdversarialScenario.ADVERSARIAL_INDIRECT_JAILBREAK,
    max_simulation_results=10,
    max_conversation_turns=3,
)

In [ ]:
pprint(unfiltered_indirect_attack_outputs)

In [ ]:
# Results are truncated for brevity.
truncation_limit = 50
for output in unfiltered_indirect_attack_outputs:
    for turn in output["messages"]:
        content = turn["content"]
        if isinstance(content, dict):  # user response from callback is dict
            print(f"{turn['role']} : {content['content'][0:truncation_limit]}")
        elif isinstance(content, tuple):  # assistant response from callback is tuple
            print(f"{turn['role']} : {content[0:truncation_limit]}")

In [ ]:
from pathlib import Path

print(unfiltered_indirect_attack_outputs)
print(unfiltered_indirect_attack_outputs.to_eval_qr_json_lines())
output = unfiltered_indirect_attack_outputs.to_eval_qr_json_lines()
xpia_file_path = "unfiltered_indirect_attack_outputs.jsonl"

# Write the output to the file
with Path.open(Path(xpia_file_path), "w") as file:
    file.write(output)

In [ ]:
indirect_attack_eval = IndirectAttackEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
file_path = "indirect_attack_outputs.jsonl"
result = evaluate(
    data=xpia_file_path,
    evaluators={
        "indirect_attack": indirect_attack_eval,
    },

    azure_ai_project=azure_ai_project,

    output_path="./mynewindirectattackevalresults.json",
)

In [ ]:
filtered_indirect_attack_outputs = await indirect_attack_simulator(
    target=xpia_callback,  # now with the Prompt Shield attached to our model deployment
    scenario=AdversarialScenario.ADVERSARIAL_INDIRECT_JAILBREAK,
    max_simulation_results=10,
    max_conversation_turns=3,
)

In [ ]:
print(filtered_indirect_attack_outputs)
print(filtered_indirect_attack_outputs.to_eval_qr_json_lines())
output = filtered_indirect_attack_outputs.to_eval_qr_json_lines()
xpia_file_path = "filtered_indirect_attack_outputs.jsonl"

# Write the output to the file
with Path.open(Path(xpia_file_path), "w") as file:
    file.write(output)

In [ ]:
filtered_indirect_attack_result = evaluate(
    data=xpia_file_path,
    evaluators={"indirect_attack": indirect_attack_eval},

    azure_ai_project=azure_ai_project,

    output_path="./myindirectattackevalresults.json",
)